# OWL tutorial

First install owlready2 if you don't already have it, and have a quick look at the [documentation](https://owlready2.readthedocs.io/en/v0.36/onto.html#).

In [24]:
## Uncomment if you do not have owlrl installed (you should have it installed from the RDFS tutorial)
#import sys
#!{sys.executable} -m pip install rdflib  owlready2 pandas

import pandas as pd
from rdflib import Graph, Literal, Namespace, RDF, URIRef, OWL
from rdflib.namespace import DC, FOAF

from owlready2 import *

Let's start loading some data from a .CSV file. We are going to create an ontology that describes the data inside.
We already did part of this using the semantics of RDF(S), now we'll use the semantics of [OWL](https://www.w3.org/TR/2012/REC-owl2-primer-20121211/) through owlready2. 

Remember that an ontology is often an application ontology, meaning that it is built with a specific task in mind. 
We could model _everything_ within a certain domain in the most ontologically correct way possible, _or_ **we could model the domain in accordance with the application's task.** 


**Your task and domain:** You are a broadcaster that has just digitised its radio archives into a digital music archive (DMA), and aims to play more interesting tracks by discovering their 'hidden treasures', by making unexpected and potentially interesting relations between tracks visible to the users (which are journalists and program makers).


**Exercise 1** 

1. look at the .csv files in the folder /data/musicoset_metadata/ and load them into pandas dataframes (use display.max_columns to show all columns). 
2. initialise an empty ontology using owlready2
3. using owlready2, create a hierarchy of classes and subclasses that describe the entities in your dataframes
4. using owrleady2, create properties and subproperties their properties, and how the classes relate to one another (using domain and range). If it helps: draw out your ontology in https://app.diagrams.net/
    - create: object properties, data properties, functional properties
5. using owlready2, add class restrictions
6. create invididuals of your classes, and provide them with attributes using your properties! 
7. write simple queries to retrieve your individuals following: https://owlready2.readthedocs.io/en/v0.36/onto.html#simple-queries. What kind of things would journalists and program makers like to retrieve? 
6. save your asserted owl file

In [25]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)


album_df = pd.read_csv("./data/musicoset_metadata/albums.csv", "\t", lineterminator="\r")

artist_df = pd.read_csv("./data/musicoset_metadata/artists.csv", "\t")
releases_df = pd.read_csv("./data/musicoset_metadata/releases.csv", "\t")
songs_df = pd.read_csv("./data/musicoset_metadata/songs.csv", "\t")
tracks_df = pd.read_csv("./data/musicoset_metadata/tracks.csv", "\t")


print("Albums")
print(album_df)

# print("Artists")
# print(artist_df) 

# print("Releases")
# print(releases_df)

# print("Songs")
# print(songs_df)


# print("Tracks")
# print(tracks_df)


Albums
                       album_id                                               name                                       billboard                                            artists  popularity  total_tracks   album_type                                          image_url
0      \n5n1GSzC1Reao29ScnpLYqp                                      Dying To Live                                   Dying To Live          {'46SHBwWsqBkxI7EeeBEQG7': 'Kodak Black'}        83.0          16.0        album  https://i.scdn.co/image/db2133234d458f432ca207...
1      \n6UYZEYjpN1DYRW0kqFy9ZE                                      Championships                                   Championships            {'20sxb77xiYeusSH8cVdatc': 'Meek Mill'}        85.0          19.0        album  https://i.scdn.co/image/77eb7c17cafe5503c58661...
2      \n7uVimUILdzSZG4KKKWToq0                 Christmas (Deluxe Special Edition)                                       Christmas        {'1GxkXlMwML1oSg5eLPiAz3': 'Michael Bub

In [26]:
onto = get_ontology("http://www.music.org/onto/")

with onto:

    # Class definitions
    class Album(Thing):
        pass

    class Song(Thing):
        pass

    class Artist(Thing):
        pass
 
    # Subclass definitions
    class Solo(Song):
        pass

    class Collab(Song):
        pass



In [27]:

with onto:
    # Album props
    class numberTracks(Album >> int):
        pass

    class releasedAlbum(Album >> str):
        pass

    class albumImage(Album >> str):
        pass

    class contains(Album >> Song):
        pass

    class createdBy(Album >> Artist):
        pass

    class albumName(DataProperty):
        domain = [Album]
        range = [str]

    class releasedDay(releasedAlbum):
        domain = [Album]
        range = [str]

    class releasedMonth(releasedAlbum):
        domain = [Album]
        range = [str]

    class releasedYear(releasedAlbum):
        domain = [Album]
        range = [str]
        


    # Artist props
    class createsSong(Artist >> Song):
        pass 
    
    class artistPopularity(Artist >> int, FunctionalProperty):
        pass

    class playsGenre(Artist >> str):
        pass

    class artistName(Artist >> str):
        pass


    # Song props 
    class trackNumber(Song >> int):
        pass

    class songName(DataProperty, FunctionalProperty):
        domain = [Song]
        range = [str]

    class songPopularity(DataProperty, FunctionalProperty):
        domain = [Song]
        range = [int]

    class isExplicit(DataProperty, FunctionalProperty):
        pass 

    class collabsWith(ObjectProperty):
        domain = [Collab]
        range = [Artist]
        pass
    
    class containedInAlbum(Song >> Album):
        inverse_property = contains




In [28]:
# Class restrictions
Artist.is_a.append(artistName.max(1))
Artist.is_a.append(createsSong.min(1, Song))
Artist.is_a.append(playsGenre.min(1))

Collab.is_a.append(collabsWith.min(1, Artist))

Song.is_a.append(albumName.max(1, int))
Song.is_a.append(trackNumber.exactly(1, int))

Album.is_a.append(contains.min(1, Song))


AllDisjoint([Solo, Collab], onto)

AllDisjoint([onto.Solo, onto.Collab])

In [29]:
# Creating instances

stronger = Song(name="Stronger", trackNumber=[2], songPopularity=11, isExplicit=False)
in_the_middle = Collab(name="She's_Like_A_Star", trackNumber=[15], songPopularity=54, isExplicit=False, collabsWith=[Artist("Taio_Cruz")])
sugababes = Artist(name="Sugababes", artistPopularity=75, playsGenre=['pop', 'r&b'], createsSong = [stronger, in_the_middle], artistName=["Sugababes"])
sugababes_album = Album(name="Angels_With_Dirty_Faces", contains=[Song('Stronger')])

jon_hopkins = Artist(name="Jon_Hopkins", artistPopularity=95, playsGenre=['dance', 'electro', 'heavy metal'], createsSong = [Song(name='Small_Memory')], artistName=["Jon_Hopkins"])
welcome = Song(name="Welcome", trackNumber=[1], songPopularity=62, isExplicit=False)
arriving = Collab(name="Arriving", trackNumber=[8], songPopularity=87, isExplicit=False, collabsWith=[Artist("7RAYS")])

mfpt_album = Album(name="Music_For_Psychedelic_Therapy", contains=[welcome, arriving], releasedMonth=["jan/21"], albumImage=["https://media.pitchfork.com/photos/612ffc02f64ed03c58cd3c34/1:1/w_320,c_limit/jonhopkins_musicforpsychedelic_3000.jpeg"], numberTracks=[9], createdBy=[jon_hopkins])







In [30]:
# Some simple queries:
print("Artists:", Artist.instances())
print("Songs:", Song.instances())
print("Subclasses of Songs:", list(Song.subclasses()))
print("Sugababes genres:", sugababes.playsGenre)

print(onto.Taio_Cruz.collabsWith)
print(onto.Jon_Hopkins.createsSong)


onto.save("./data/firstOwlOnto.owl", format="rdfxml")


Artists: [onto.Taio_Cruz, onto.Sugababes, onto.Jon_Hopkins, onto.7RAYS]
Songs: [onto.Stronger, onto.Small_Memory, onto.Welcome, onto.She's_Like_A_Star, onto.Arriving]
Subclasses of Songs: [onto.Solo, onto.Collab]
Sugababes genres: ['pop', 'r&b']
[]
[onto.Small_Memory]


## OWL reasoning 

Let's look at how reasoning works.

Owlready automatically gets the results of the reasoning from HermiT (a type of reasoner) and reclassifies Individuals and Classes. 

**Exercise 2**
1. think about which things are inferred from your OWL semantics. Query/look at your graph: do you see what you expected?
2. looking at the following tutorial [owlready2-reasoning](https://owlready2.readthedocs.io/en/latest/reasoning.html), which things have not yet been inferred? Run the owlready2 reasoner to:
    - infer these new triples
    - check your ontology and statements (individuals + attributes) for consistency
3. save your asserted + inferred triples to a new file 

In [33]:
with onto:
    sync_reasoner(infer_property_values=True)

print(onto.Jon_Hopkins.createsSong)
onto.save("./data/inferred_asserted_triples.owl", format="rdfxml")


* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /home/ruben/.local/lib/python3.8/site-packages/owlready2/hermit:/home/ruben/.local/lib/python3.8/site-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmpwcan03l4 -Y


[onto.Small_Memory]


* Owlready2 * HermiT took 0.6586036682128906 seconds
* Owlready * (NB: only changes on entities loaded in Python are shown, other changes are done but not listed)


#### Querying inferred triples

**Exercise 3**
Query your inferred triples: 

- *.get_parents_of(entity)* accepts any entity (Class, property or individual), and returns the superclasses (for a class), the superproperties (for a property), or the classes (for an individual). 

- *.get_instances_of(Class)* returns the individuals that are asserted as belonging to the given Class in the ontology. (NB for obtaining all instances, independently of the ontology they are asserted in, use Class.instances()).

- *.get_children_of(entity)* returns the subclasses (or subproperties) that are asserted for the given Class or property in the ontology. (NB for obtaining all children, independently of the ontology they are asserted in, use entity.subclasses()).